In [ ]:
# | default_exp _cli_docs

In [ ]:
# | export

import asyncio
import platform
import signal
import socketserver
from http.server import SimpleHTTPRequestHandler
from pathlib import Path
from types import FrameType
from typing import *

import typer

from fastkafka._components.docs_dependencies import (
    _check_npm_with_local,
    _install_docs_npm_deps,
    _install_node,
)
from fastkafka._components.helpers import _import_from_string, change_dir
from fastkafka._components.logger import get_logger

In [ ]:
import time

from typer.testing import CliRunner

from fastkafka._components.logger import suppress_timestamps
from fastkafka._components.test_dependencies import generate_app_in_tmp
from fastkafka._server import terminate_asyncio_process

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
runner = CliRunner()

In [ ]:
# | export

_docs_app = typer.Typer(help="Commands for managing fastkafka app documentation")

In [ ]:
# | export


@_docs_app.command(
    "install_deps",
    help="Installs dependencies for FastKafka documentation generation",
)
def docs_install_deps() -> None:
    """
    Installs dependencies for FastKafka documentation generation.

    Raises:
        typer.Abort: If the user chooses not to install NodeJS and npm locally.
    """
    try:
        _check_npm_with_local()
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        install_confirm = typer.confirm(
            "npm not found or version is too low, do you want us to install the NodeJS and npm locally?"
        )
        if install_confirm is False:
            print("Not installing NodeJS and npm locally, exiting..")
            raise typer.Abort()
        else:
            _install_node()
    asyncio.run(_install_docs_npm_deps())


@_docs_app.command(
    "generate",
    help="Generates documentation for a FastKafka application",
)
def generate_docs(
    root_path: Optional[str] = typer.Option(
        default=None,
        help="root path under which documentation will be created; default is current directory",
        show_default=False,
    ),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    """
    Generates documentation for a FastKafka application.

    Args:
        root_path: The root path under which the documentation will be created.
            Default is the current directory.
        app: Input in the form of 'path:app', where **path** is the path to a python
            file and **app** is an object of type **FastKafka**.

    Raises:
        typer.Exit: If there is an unexpected internal error.
    """
    try:
        application = _import_from_string(app)
        if root_path is not None:
            application._root_path = Path(root_path)
            application._asyncapi_path = application._root_path / "asyncapi"

        application.skip_docs = False
        application.create_docs()
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)


@_docs_app.command(
    "serve",
    help="Generates and serves documentation for a FastKafka application",
)
def serve_docs(
    root_path: str = typer.Option(
        default=None,
        help="root path under which documentation will be created; default is current directory",
        show_default=False,
    ),
    bind: str = typer.Option("127.0.0.1", help="Some info"),
    port: int = typer.Option(8000, help="Some info"),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    """
    Generates and serves documentation for a FastKafka application.

    Args:
        root_path: The root path under which the documentation will be created.
            Default is the current directory.
        bind: The IP address to bind the server to. Default is '127.0.0.1'.
        port: The port number to bind the server to. Default is 8000.
        app: Input in the form of 'path:app', where **path** is the path to a python
            file and **app** is an object of type **FastKafka**.

    Raises:
        typer.Exit: If there is an unexpected internal error.
    """
    try:
        application = _import_from_string(app)
        if root_path is not None:
            application._root_path = Path(root_path)
            application._asyncapi_path = application._root_path / "asyncapi"

        application.create_docs()
        with change_dir(str(application._asyncapi_path / "docs")):
            server_address = (bind, port)
            handler = SimpleHTTPRequestHandler

            d = {"should_stop": False}

            def sigint_handler(
                signal: int, frame: Optional[FrameType], d: Dict[str, bool] = d
            ) -> None:
                d["should_stop"] = True

            signal.signal(signal.SIGINT, sigint_handler)
            signal.signal(signal.SIGTERM, sigint_handler)
            if platform.system() == "Windows":
                signal.signal(signal.SIGBREAK, sigint_handler) # type: ignore

            with socketserver.TCPServer(server_address, handler) as httpd:
                httpd.timeout = 0.1
                typer.secho(
                    f"Serving documentation on http://{server_address[0]}:{server_address[1]}"
                )
                while not d["should_stop"]:
                    httpd.handle_request()
                typer.secho(f"Interupting serving of documentation and cleaning up...")
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
# | notest

! nbdev_export

In [ ]:
result = runner.invoke(_docs_app, ["install_deps", "--help"])

                                                                                                                   
 Usage: root install_deps [OPTIONS]                                                                                
                                                                                                                   

Installs dependencies for FastKafka documentation generation

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --help          Show this message and exit.                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = runner.invoke(_docs_app, ["install_deps"])
assert result.exit_code == 0

[INFO] fastkafka._components.docs_dependencies: AsyncAPI generator installed


In [ ]:
result = runner.invoke(_docs_app, ["generate", "--help"])

                                                                                                                   
 Usage: root generate [OPTIONS] APP                                                                                
                                                                                                                   

Generates documentation for a FastKafka application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT  root path under which documentation will be created [default: .]                       │
│ --help                   Show this message and exit.                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with generate_app_in_tmp() as import_str:
    result = runner.invoke(_docs_app, ["generate", import_str])
    typer.echo(result.output)
    assert result.exit_code == 0

    result = runner.invoke(_docs_app, ["generate", import_str])
    typer.echo(result.output)
    assert result.exit_code == 0

[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmpqvalk_fi/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmpqvalk_fi/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka._components.asyncapi: Async docs generated at 'asyncapi/docs'
[INFO] fastkafka._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/tmpqvalk_fi/asyncapi/docs.






In [ ]:
result = runner.invoke(_docs_app, ["serve", "--help"])

                                                                                                                   
 Usage: root serve [OPTIONS] APP                                                                                   
                                                                                                                   

Generates and serves documentation for a FastKafka application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT     root path under which documentation will be created [default: .]                    │
│ --bind             TEXT     Some info [default: 127.0.0.1]                                                      │
│ --port             INTEGER  Some info [default: 8000]                                                           │
│ --help                      Show this message and exit.                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯